<a href="https://colab.research.google.com/github/CarlosVargasF/Stage_Liris_Vargas/blob/master/Code/two_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install infomap
!pip install wurlitzer
!pip install leidenalg 
!pip install git+https://github.com/GiulioRossetti/cdlib.git > /dev/null
#!pip install cigram 

  Running command git clone -q https://github.com/GiulioRossetti/cdlib.git /tmp/pip-req-build-i49_ukz5


In [33]:
from cdlib import algorithms
#from cdlib import viz
from cdlib import NodeClustering
from cdlib import evaluation
import networkx as nx
#from networkx.generators.community import LFR_benchmark_graph
import time
import pandas as pd
import numpy as np
#from cigram import lfr_benchmark_graph
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

In [34]:
#*********************** INNER FUNC (GENERIC FUNCTION)*************************************
def evaluate(graph_comms_list, algorithm_dict, eval_method_dict, benchmark, case_name=None):
  
  if case_name:
    case = case_name
  else:
    case = '-'

  #Check graph_comms_list parameter
  if isinstance(graph_comms_list, list) and len(graph_comms_list):
    graphs = [gc[0] for gc in graph_comms_list] #-------------------update this part
    communities = [gc[1] for gc in graph_comms_list]
    attributs = [gc[2] for gc in graph_comms_list]
  elif isinstance(graph_list, tuple):
    graphs = list(graph_comms_list[0])
    communities = list(graph_comms_list[1])
    attributs = list([gc[2] for gc in graph_comms_list])
  else:
    raise ValueError("Please insert a list of couples (graph,commnunities).")

  #Creation of NodeClustering objects     
  lfr_nodclust_obj = [NodeClustering(C, G, 'Ground Truth') for C, G in zip(communities, graphs)]

  #Application of algorithms over all the benchmarks
  res = pd.DataFrame(columns=['Algorithm', 'Graph', 'Performance', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])
     
  for alg in algorithm_dict:
    algo_data = []    
    for G, C, A in zip(graphs, lfr_nodclust_obj, attributs):
      try:
        s_time = time.time()
        solution = algorithm_dict[alg](G)
        e_time = time.time() - s_time
        for method in eval_method_dict:
          #Performance evaluation (NMI)
          perf = eval_method_dict[method](C, solution)
          algo_data.append([alg, G.name, perf[0], method, benchmark, case, e_time, 'ok', A['n'], A['sz'], A['ut'], A['rep']])
      except Exception as e:
        algo_data.append([alg, G.name, '-', '-', benchmark, case, '-', repr(e), '-','-','-','-'])
    
    #Update of results
    algo_data_df = pd.DataFrame(algo_data, columns=['Algorithm', 'Graph', 'Performance', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])
    #algo_data_df['NMI'] = perf_df['score']
    res = pd.concat([res, algo_data_df], ignore_index=True)
   
  return res

In [35]:
# *********************************** OUTTER FUNC (STANDARD TEST CASES) *******************************************

def evaluate_and_compare(algos_list, methods_list, benchmark_type, case_list='all'):
  '''
  Some CD algorthms (CDlib) <algos_list>       [CD algorithms to apply] :
    - 'gn'            [girvan_newman]
    - 'greedy'        [greedy_modularity (Clauset et al)]
    - 'louvain'       [louvain (Blondel et al)]
    - 'cfinder'       [kclique]
    - 'mcl'           [markov_clustering]
    - 'infomap'       [infomap]
    - 'em'            [expectation-maximization]
    - 'leiden'        [leiden]
  
  Partition evaluation methods (CDlib) <methods_list>       [evaluation metric to apply] :
    - 'nmi'           [girvan_newman]
    - 'ami'           [greedy_modularity (Clauset et al)]
    - 'ari'           [louvain (Blondel et al)]
    - 'f1'            [kclique]
    - 'nf1'           [markov_clustering]
    - 'omega'         [infomap]
    - 'onmi_lfk'      [expectation-maximization]
    - 'onmi_mgh'      [leiden]
    - 'var_inf'       [leiden]

  Possible values for <benchmark_type>       [benchmark to evaluate] :
    - 'lfr_undir_unwei'
    - 'lfr_undir_wei'
    - 'lfr_dir_unwei'
    - 'lfr_undir_unwei_ovlp'

  Possible values for <case_list>       [benchmark cases to evaluate] :
    - 'n_1k_small'     [n=1000, minc=10, maxc=50]
    - 'n_1k_big'       [n=1000, minc=10, maxc=50] 
    - 'n_5k_small'     [n=5000, minc=20, maxc=100]
    - 'n_5k_big'       [n=5000, minc=20, maxc=100]
    - 'all'           [perform evaluation over all 4 cases]

    * It is possible to select multiple cases passing them as a list.
  '''
  
  start_time = time.time()
  
  #CD Algorithms names list (Crisp Communities)
  algos_dict = {
    'gn'          : algorithms.girvan_newman,
    'greedy'      : algorithms.greedy_modularity,
    'louvain'     : algorithms.louvain,
    'cfinder'     : algorithms.kclique,
    'mcl'         : algorithms.markov_clustering,
    'infomap'     : algorithms.infomap,
    'em'          : algorithms.em,
    'leiden'      : algorithms.leiden,

    'agdl'        : algorithms.agdl,
    'aslpaw'      : algorithms.aslpaw,
    'async_fluid' : algorithms.async_fluid,
    'cpm'         : algorithms.cpm,
    'der'         : algorithms.der,
    'edmot'       : algorithms.edmot,
    'eigenvector' : algorithms.eigenvector,
    'gdmp2'       : algorithms.gdmp2,
    'label_prop'  : algorithms.label_propagation,
    'rber_pots'   : algorithms.rber_pots,
    'rb_pots'     : algorithms.rb_pots,
    'scan'        : algorithms.scan,
    'signif_com'  : algorithms.significance_communities,
    'spinglass'   : algorithms.spinglass,
    'surp_com'    : algorithms.surprise_communities,
    'walktrap'    : algorithms.walktrap,
    'sbm_dl'      : algorithms.sbm_dl,
    'sbm_dl_nstd' : algorithms.sbm_dl_nested       
  }

  #Partition comparisons scores (CDlib)
  methods_dict = {
    'nmi'      : evaluation.normalized_mutual_information,
    'ami'      : evaluation.adjusted_mutual_information,
    'ari'      : evaluation.adjusted_rand_index,
    'f1'       : evaluation.f1,
    'nf1'      : evaluation.nf1,
    'omega'    : evaluation.omega,
    'onmi_lfk' : evaluation.overlapping_normalized_mutual_information_LFK,
    'onmi_mgh' : evaluation.overlapping_normalized_mutual_information_MGH,
    'vi'  : evaluation.variation_of_information
  }

  #Importing and reading benchmark files

  #Url base path
  url = 'https://github.com/CarlosVargasF/LFR_benchmarks_for_testing/raw/bf96970a72de0707fdb7629295d7644f76d83ae8/'
  
  ''' #Folders for undirected and unweighted benchmarks
  src_undir_unwei = {
    'n_1k_sz_small' : 'lfr_n1000_small/',
    'n_1k_sz_big'   : 'lfr_n1000_big/',
    'n_5k_sz_small' : 'lfr_n5000_small/',
    'n_5k_sz_big'   : 'lfr_n5000_big/'
  }

  #Folders for undirected and weighted benchmarks
  src_undir_wei = {
    'n_5k_sz_small_mut05' : 'lfr_n5000_small_mut05/',
    'n_5k_sz_small_mut08' : 'lfr_n5000_small_mut08/',
    'n_5k_sz_big_mut05'   : 'lfr_n5000_big_mut05/',
    'n_5k_sz_big_mut08'   : 'lfr_n5000_big_mut08/'
  }

   #Folders for directed and unweighted benchmarks
  src_dir_unwei = {
    'n_1k_sz_small' : 'lfr_n1000_small/',
    'n_1k_sz_big'   : 'lfr_n1000_big/',
    'n_5k_sz_small' : 'lfr_n5000_small/',
    'n_5k_sz_big'   : 'lfr_n5000_big/'
  }

  #Folders for undirected and weighted benchmarks
  src_undir_wei = {
    'n_5k_sz_small_mut01' : 'lfr_n5000_small_mut01/',
    'n_5k_sz_small_mut03' : 'lfr_n5000_small_mut03/',
    'n_5k_sz_big_mut01'   : 'lfr_n5000_big_mut01/',
    'n_5k_sz_big_mut03'   : 'lfr_n5000_big_mut03/'
  } '''

  #Graph type
  if isinstance(benchmark_type, str) and len(benchmark_type):
    if benchmark_type == 'undir_unwei':
      url = url + 'undirected_unweighted/lfr_'
      #src = src_undir_unwei
    elif benchmark_type == 'undir_wei':
      url = url + 'undirected_weighted/lfr_'
      #src = src_undir_wei
    elif benchmark_type == 'dir_unwei':
      url = url + 'directed_unweighted/lfr_'
      #src = src_dir_unwei
    else:
      raise ValueError('benchmark_type not supported')
  else:
    raise TypeError('Insert a valide benchmark_type as a string')

  #Checking case parameter  
  if isinstance(case_list, str): 
    if case_list == 'all':
      cases = [opt for opt in src]
    else:
      cases = list(case_list.split())
  elif (isinstance(case_list, list)) and (len(case_list)): 
    for c in case_list:
      if not(c in src):
        raise ValueError('Invalid <case> option. Verify available cases for the selected benchmark.')
    cases = case_list    
  else:
    raise TypeError('Invalid <case> input format. Please insert a list or a space-separated string of valide options or "all" for use all of them.')
  
  #Checking algos_list parameter
  if (isinstance(algos_list, str)):
    algos = dict((k, v) for k, v in algos_dict.items() if k in list(algos_list.split()))
  elif (isinstance(algos_list, list)) and (len(algos_list)): 
    for a in algos_list:
      if not(a in algos_dict):
        raise ValueError('Invalid algos_dict key. Verify available algorithm names.')
    algos = dict((k, v) for k, v in algos_dict.items() if k in algos_list)    
  else:
    raise TypeError('Invalid <algos_list> input format. Please insert a list or a space-separated string of valide algorithm names.') 

  #Checking methods_list parameter
  if (isinstance(methods_list, str)):
    methods = dict((k, v) for k, v in methods_dict.items() if k in list(methods_list.split()))
  elif (isinstance(methods_list, list)) and (len(methods_list)): 
    for m in methods_list:
      if not(m in methods_dict):
        raise ValueError('Invalid methods_dict key. Verify available evaluation method names.')
    methods = dict((k, v) for k, v in methods_dict.items() if k in methods_list)    
  else:
    raise TypeError('Invalid <methods_list> input format. Please insert a list or a space-separated string of evaluation method names.') 

  #Generate graphs according to cases   
  results = pd.DataFrame(columns=['Algorithm', 'Graph', 'Performance', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])
  for case in cases:
    base_path = url + case 

    #Open zip files
    zip_comms = urlopen(base_path + '/cnl_files.zip')
    zip_edges = urlopen(base_path + '/nse_files.zip')
    zipfile_c = ZipFile(BytesIO(zip_comms.read()))
    zipfile_e = ZipFile(BytesIO(zip_edges.read()))

    #Read data files
    data_comms = [zipfile_c.open(line1).read().decode('utf-8').splitlines() for line1 in zipfile_c.namelist()]
    data_edges = [(zipfile_e.open(line2), line2) for line2 in zipfile_e.namelist()]
        
    #Construction of the lfr graphs
    lfr_graphs = []
    graph_attributes = []
    for edge_file in data_edges:
      attrib_dict = {}
      g = nx.read_edgelist(edge_file[0], nodetype=int, data=False)
      g.name = edge_file[1]

      attribs_1 = case.split('_')
      attribs_2 = edge_file[1].split('_')
      
      attrib_dict.update([('name', edge_file[1][:-4]),('n',attribs_1[attribs_1.index('n')+1]), ('sz',attribs_1[attribs_1.index('sz')+1]),
                          ('ut',attribs_2[attribs_2.index('ut')+1]), ('rep',attribs_2[attribs_2.index('rep')+1][:-4])])
      lfr_graphs.append(g)
      graph_attributes.append(attrib_dict)

    #Construction of the lfr communities
    lfr_comms = [[list(map(int, item.split())) for item in comm_file] for comm_file in data_comms]

    #Formating for the "evaluate" function 
    graphs_comms_atts = [(g, c, a) for g, c, a in zip(lfr_graphs, lfr_comms, graph_attributes)]

   
    #----------INNER FUNC CALL--------------------------------------------------

    #Apply selected CD algorithms and evaluate them according to selected methods
    res_eval = evaluate(graphs_comms_atts, algos, methods, benchmark_type, case)

    #----------INNER FUNC CALL--------------------------------------------------

    results = pd.concat([results, res_eval], ignore_index=True)

  total_time = (time.time() - start_time)
  print("--- Total execution time: %d min %d sec ---" % (total_time//60, total_time%60))

  return results 


In [36]:
p=evaluate_and_compare('infomap', 'nmi', 'undir_unwei', 'n_1k_sz_small')
p

--- Total execution time: 0 min 17 sec ---


,Algorithm,Graph,Performance,Metric,Benchmark,Case,Exec_Time,Status,N,Size_C,ut,Rep
0,infomap,lfr_ut_0.0_rep_0.nse,9.655765e-01,nmi,undir_unwei,n_1k_sz_small,0.085910,ok,1k,small,0.0,0
1,infomap,lfr_ut_0.0_rep_1.nse,9.773907e-01,nmi,undir_unwei,n_1k_sz_small,0.080362,ok,1k,small,0.0,1
2,infomap,lfr_ut_0.0_rep_2.nse,9.622740e-01,nmi,undir_unwei,n_1k_sz_small,0.082477,ok,1k,small,0.0,2
3,infomap,lfr_ut_0.0_rep_3.nse,9.454684e-01,nmi,undir_unwei,n_1k_sz_small,0.088539,ok,1k,small,0.0,3
4,infomap,lfr_ut_0.0_rep_4.nse,9.617126e-01,nmi,undir_unwei,n_1k_sz_small,0.083631,ok,1k,small,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
105,infomap,lfr_ut_1.0_rep_5.nse,1.124139e-17,nmi,undir_unwei,n_1k_sz_small,0.108293,ok,1k,small,1.0,5
106,infomap,lfr_ut_1.0_rep_6.nse,-5.805019e-17,nmi,undir_unwei,n_1k_sz_small,0.082499,ok,1k,small,1.0,6
107,infomap,lfr_ut_1.0_rep_7.nse,-1.170697e-16,nmi,undir_unwei,n_1k_sz_small,0.088151,ok,1k,small,1.0,7
108,infomap,lfr_ut_1.0_rep_8.nse,-8.621260e-17,nmi,undir_unwei,n_1k_sz_small,0.100709,ok,1k,small,1.0,8
